# Introduction
## Part 2: Quantum Chemistry

In this tutorial, you will learn how to run basic quantum chemistry
calculations.
Quantum chemistry uses quantum mechanics to predict chemical properties.

### Potential energy surfaces

A fundamental simplifying assumtion in most (not all) quantum chemistry
calculations is the
[Born-Oppenheimer approximation](https://en.wikipedia.org/wiki/Born%E2%80%93Oppenheimer_approximation),
which assumes that the electrons in a molecule move so quickly that they
rearrange instantaneously as the nuclei move.
This is sometimes described as the "flies on an ox model."
This approximation separates the molecular
[Schrödinger equation](https://en.wikipedia.org/wiki/Schr%C3%B6dinger_equation)
into two simpler equations:
$$
\begin{align*}
    \hat{H}_\text{e}(\mathbf{r}_\text{n})
    \Psi_\text{e}(\mathbf{r}_\text{e}; \mathbf{r}_\text{n})
    =&
    E_\text{e}(\mathbf{r}_\text{n})
    \Psi_\text{e}(\mathbf{r}_\text{e}; \mathbf{r}_\text{n})
    \\
    (\hat{T}_\text{n} + E_\text{e}(\mathbf{r}_\text{n}))
    \Psi_\text{n}(\mathbf{r}_\text{n})
    =& E_\text{n}
    \Psi_\text{n}(\mathbf{r}_\text{n})
\end{align*}
$$

If you have not taken a physical chemistry or quantum mechanics course, these
equations may not make much sense to you.
For our purposes, the important point is that the total energy of the
electrons is a function of the nuclear coordinates,
$E_\text{e}(\mathbf{r}_\text{n})$,
and that this function defines the *potential energy* of the nuclei.
That is, these electronic energies comprise the *potential energy surface* (PES)
that governs how the atoms in a molecule move and react.
The concept of the PES is fundamental to all of quantum chemistry.

Through the Born-Oppenheimer approximation, quantum chemistry becomes a two-step
process:
1. The first step is to calculate PES values for the relevant molecular
structures. These are called *electronic structure calculations* because they
solve for the state of the electrons. Sometimes they are also called
*single-point calculations* because they give us information about one point on
the PES.
2. Use these energies to determine properties of the molecule or reaction of interest.

To see how this connects with reality, consider a reaction energy diagram:
![reaction energy diagram](../../.github/chemistry-energy-diagram.svg)

This diagram represents a slice of the PES for a reaction.
If we can determine the sequence of molecular structures or "geometries" between
the reactants and the products of this reaction, we can generate the above
energy diagram through electronic structure calculations on each of these
points.
In fact, this is exactly the kind of thing we can do to calculate the rate of a
reaction using quantum chemistry.

### The QC Suite of Programs

There are many quantum chemistry codes out there, each with their own syntax for
invoking a calculation and printing the results.
To mitigate this complexity, we will be making use of the QC suite of programs
developed by Colton Hicks, which allows us to operate such programs through a
common interface.
This tutorial will introduce you to two of the modules that you will be
interacting with in the QC suite:
- [qcio](https://github.com/coltonbh/qcio) standardizes inputs and outputs to quantum chemistry programs. Documentation [here](https://qcio.coltonhicks.com/).
- [qcop](https://github.com/coltonbh/qcop) runs quantum chemistry programs to calculate results. Documentation [here](https://qcop.coltonhicks.com/).

Our `protomol` helper module already contains a submodule,
[qc.struc](../../src/protomol/qc/struc.py), for working with the `Structure`
objects used by `qcio`.

The following code shows how to go from a SMILES string to a `qcio.Structure` object.

In [1]:
from protomol import smiles, qc

smi = "C[CH2]"
geo = smiles.geometry(smi)
struc = qc.struc.from_geometry(geo)
qc.struc.display(struc)

Next, we can use `qcop` to calculate the single-point electronic energy of our RDKit-generated starting structure.

In [2]:
import qcio
import qcop

# Define the calculation
prog_inp = qcio.ProgramInput(
    structure=struc, calctype="energy", model={"method": "gfn2"}
)

# Run the calculation
prog_out = qcop.compute("crest", prog_inp)

# Gather the results
energy = prog_out.results.energy
print("Energy:", energy)

Energy: -6.725683011794641


Under the hood, this is using the quantum chemistry program
[CREST](https://crest-lab.github.io/crest-docs/)
to do the energy calculation, using the
[GFN2-xTB](https://pubs.acs.org/doi/10.1021/acs.jctc.8b01176)
electronic structure method.
The energy is given in [atomic units](https://en.wikipedia.org/wiki/Atomic_units) ([Hartree](https://en.wikipedia.org/wiki/Hartree)).
This gives us a point on the PES of ethyl radical, but not a very relevant one, because RDKit only gives us a chemically reasonable starting structure, not an equilibrium structure.

To determine this molecule's *equilibrium structure*, we need to perform a structure optimization on the PES to find a local minimum.
This can be done as follows.

In [12]:
# Define the calculation
prog_inp = qcio.ProgramInput(
    structure=struc, calctype="optimization", model={"method": "gfn2"}
)

# Run the calculation
prog_out = qcop.compute("crest", prog_inp)

# Gather the results
final_energy = prog_out.results.final_energy
print("Energy (Hartree):", final_energy)
print("Energy change (Hartree):", final_energy - energy)

Energy (Hartree): -6.7352941698
Energy change (Hartree): -0.0096111580053595


Notice that the final energy after optimization is now lower (more negative).
We can use
[Pint](https://pint.readthedocs.io/en/stable/getting/tutorial.html)
to calculate this energy difference in more familiar units.

In [11]:
from pint import Quantity

to_kcal = Quantity("hartree/molecule").m_as("kcal/mole")
to_kj = Quantity("hartree/molecule").m_as("kJ/mole")

print("Energy change (kcal/mole):", (final_energy - energy) * to_kcal)
print("Energy change (kJ/mole):", (final_energy - energy) * to_kj)

Energy change (kcal/mole): -6.031092705080164
Energy change (kJ/mole): -25.23409187805541


### Next step: move on to the first project!

There is no exercise associated with this tutorial. You will be practicing the things you learned about here in the first programming project.